__load data

In [1]:
import pandas as pd
df = pd.read_csv("ML tutorial data (county income).csv", sep=',', encoding='latin-1')

In [2]:
list(df)[0:25]

['GEOID',
 'INTPTLAT',
 'INTPTLONG',
 'Percentage_whites',
 'Percentage_blacks',
 'Median_Income',
 'Percent_No_HighSchool',
 'Percent_HighSchool_only',
 'Percent_Some_College',
 'Percent_Bachelor_or_higher',
 'crime_per_person',
 'population_size',
 'Ratio_Employed',
 'percent_democrat_votes2016',
 'percent_republican_votes2016',
 'voter_participation',
 'sex_ratio',
 'median_age',
 'percent_democrat_votes2012',
 'percent_republican_votes2012',
 'tblack',
 'twhite',
 'X..12',
 '.',
 'the']


__choose predictors and outcome (also preprocessing)


In [3]:
predictors = df.loc[:, "Percent_No_HighSchool" : "Percent_Bachelor_or_higher"]
outcome = df.loc[:,"Median_Income"]


__choose a model


In [6]:
from sklearn.linear_model import LinearRegression
Regr = LinearRegression()


__fit and cross validate model (easy way without hyperparameters)


In [7]:
from sklearn.model_selection import cross_val_score
import numpy as np

results = cross_val_score(Regr, predictors, outcome, cv = 10, scoring = "r2")
print(results)
print(np.mean(results))


[ 0.54763227  0.23312451  0.38968369  0.50994311 -0.01115834  0.46640691
  0.41416274  0.46438321  0.43303891  0.44613911]
0.389335611706



__Now, Let's try something else

__New concepts:
 We use a decision tree.
 We tune the hyperparameter "max_depth"

__First step: put aside some test data

In [8]:
from sklearn.model_selection import GridSearchCV, train_test_split

predictors = df.loc[:, ["Ratio_Employed", "population_size", "percent_democrat_votes2016", "percent_republican_votes2016"]]
outcome = df.loc[:,"Median_Income"]

predictors_train, predictors_test, outcome_train, outcome_test = train_test_split(predictors, outcome, test_size=300, random_state=42)



__Give possible values for hyperparameters


In [9]:
hyperparameters = {"max_depth": [2,3,4,None]}

__fit the decision tree with all given parameter settings

In [10]:
from sklearn import tree

decTree = tree.DecisionTreeRegressor() #inititate

decTree_cv = GridSearchCV(decTree, hyperparameters, cv=10) #set up the search for the best parameter

decTree_cv.fit(predictors_train, outcome_train) #fit the competing models

GridSearchCV(cv=10, error_score='raise',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': [2, 3, 4, None]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)


__what is the best model/settings


In [11]:
print(decTree_cv.best_score_)
print(decTree_cv.best_params_)

0.491053590469
{'max_depth': 4}



__and with more detail...


In [13]:
for parameter, accuracy in zip(decTree_cv.cv_results_['params'], decTree_cv.cv_results_['mean_train_score']):
    print(parameter)
    print(accuracy)

{'max_depth': 2}
0.426852965332
{'max_depth': 3}
0.513738985284
{'max_depth': 4}
0.57108499148
{'max_depth': None}
1.0



__make predictions for new sample


In [14]:
outcome_pred = decTree_cv.predict(predictors_test)

from sklearn.metrics import r2_score
print(r2_score(outcome_test, outcome_pred))


0.486299140201


plot the tree 

In [15]:
from sklearn.tree import export_graphviz

dotfile = open("dtree.dot", 'w')
tree.export_graphviz(decTree_cv.best_estimator_, out_file = dotfile, feature_names = ["Ratio_Employed", "population_size", "percent_democrat_votes2016", "percent_republican_votes2016"])
dotfile.close()

#http://webgraphviz.com/

__try different models from the lecture before

1) random forest: http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

2) k nearest neighbors: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html

3) ridge regression: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

In [16]:
#choose predictors and outcome



In [ ]:
#split off a training and test set...



In [ ]:
#maybe standardize variables?

from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
predictors_train_std=sc.fit_transform(predictors_train)
predictors_test_std=sc.transform(predictors_test)